In [48]:
library(ggplot2)
library(numbat)
library(dplyr)
library(glue)
library(data.table)
library(ggtree)
library(stringr)
library(tidygraph)
library(patchwork)

In [49]:
options(repr.plot.width = 14, repr.plot.height = 8)

In [50]:
plot_theme = theme(title = element_text(size = 24),
                   axis.title = element_text(size = 20),
                   axis.text = element_text(size = 16),
                   legend.text = element_text(size = 16))

### 20230228 check run numbat on p009

gtf_hg38.rda ##
https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/genes/hg38.refGene.gtf.gz

In [51]:
getwd()

[1] "/fast/work/projects/scCRC_Patients/scitcem/datasets/numbat"

In [52]:
donor = 'p009'

all_counts <- readRDS('../anno/CRC/3p/all_epi_counts.rds')

In [53]:
all_counts = readRDS('../../datasets_new_preprocessing/all_epi_cell_CB_counts.rds')

In [54]:
dim(all_counts) #dgCMatrix col:p007t_AACAACCAGTTGTCGT, row: gene name

[1] 33538 22460

In [55]:
head(colnames(all_counts))

[1] "p014n:AAACCCAAGCAAATGT" "p014n:AAACCCAAGTACCGGA" "p014n:AAACCCACAGGGAATC"
[4] "p014n:AAACCCACATACCAGT" "p014n:AAACCCAGTTTGCAGT" "p014n:AAACCCATCATCGTAG"

In [56]:
tumor_counts <- all_counts[,grepl(paste0(donor,'t'),colnames(all_counts))]

In [57]:
dim(tumor_counts)

[1] 33538  3385

In [58]:
colnames(tumor_counts) %>% head()

[1] "p009t2:AAACCCAAGGTAGATT" "p009t2:AAACCCAGTCAGCTTA"
[3] "p009t2:AAACCCAGTCCTTTGC" "p009t2:AAACGAACATCCGGCA"
[5] "p009t2:AAACGAATCCTTATGT" "p009t2:AAACGAATCTGAGATC"

In [59]:
normal_counts <- all_counts[,grepl('n',colnames(all_counts))] # ? # only p009?

In [60]:
dim(normal_counts)

[1] 33538 13141

In [61]:
allele_df <- read.table(paste0('../../datasets_new_preprocessing/Numbat/',donor,'/',donor,'t1_allele_counts.tsv.gz'),sep='\t',header=1)


In [62]:
head(allele_df$cell)

[1] "ATTACCTAGATTGAGT-1" "CTCAATTCAAGACGAC-1" "GAACTGTCATGGCCCA-1"
[4] "GGTCACGGTCACAGTT-1" "GTATTGGGTCCAACGC-1" "TCATTTGCACACTGGC-1"

In [63]:
allele_df$cell <- paste0(donor,'t1:',gsub('-1$','',allele_df$cell))


In [64]:
dim(allele_df)

[1] 1945968      11

In [65]:
head(allele_df)

,cell,snp_id,CHROM,POS,cM,REF,ALT,AD,DP,GT,gene
,<chr>,<chr>,<int>,<int>,<dbl>,<chr>,<chr>,<int>,<int>,<chr>,<chr>
1,p009t1:ATTACCTAGATTGAGT,1_630110_T_C,1,630110,0,T,C,0,1,0|1,
2,p009t1:CTCAATTCAAGACGAC,1_630110_T_C,1,630110,0,T,C,0,1,0|1,
3,p009t1:GAACTGTCATGGCCCA,1_630110_T_C,1,630110,0,T,C,0,1,0|1,
4,p009t1:GGTCACGGTCACAGTT,1_630110_T_C,1,630110,0,T,C,1,1,0|1,
5,p009t1:GTATTGGGTCCAACGC,1_630110_T_C,1,630110,0,T,C,0,1,0|1,
6,p009t1:TCATTTGCACACTGGC,1_630110_T_C,1,630110,0,T,C,0,1,0|1,


In [66]:
allele_df2 <- read.table(paste0('../../datasets_new_preprocessing/Numbat/',donor,'/',donor,'t2_allele_counts.tsv.gz'),sep='\t',header=1)


In [67]:
allele_df2$cell <- paste0(donor,'t2:',gsub('-1$','',allele_df2$cell))


In [68]:
dim(allele_df2)

[1] 2260535      11

In [69]:
anno <- read.csv('../../datasets_new_preprocessing/all_epi_cell_anno.txt',sep=' ',header=1)
anno = anno %>% tibble::rownames_to_column('cell')
colnames(anno) = c('cell', 'group')

In [71]:
table(anno['group'])


             epi  malignant_p007t  malignant_p008t malignant_p009t1 
           21833             1474              704             3057 
malignant_p009t2  malignant_p013t  malignant_p014t  malignant_p016t 
            3009             1872             1840             1524 
 malignant_p020t  malignant_p021t  malignant_p026t  malignant_p035t 
            1361              351             1190             1565 

In [72]:
tumor_anno <- anno[grepl(paste0(donor,'t1'),anno$cell),]

In [73]:
head(tumor_anno)

,cell,group
,<chr>,<chr>
25724,p009t1:AATGCCATCCTAACAG,malignant_p009t1
25725,p009t1:CTTTCAATCGAGCTGC,malignant_p009t1
25726,p009t1:GCCAGTGGTCTCGCGA,malignant_p009t1
25727,p009t1:GGTCACGGTCACAGTT,malignant_p009t1
25728,p009t1:AGACACTAGTATCCTG,malignant_p009t1
25729,p009t1:TGATGGTCACTCTCGT,malignant_p009t1


In [74]:
dim(tumor_anno)

[1] 3057    2

In [81]:
tumor_anno2 <- anno[grepl(paste0(donor,'t2'),anno$cell),]
dim(tumor_anno2)
head(tumor_anno2)

[1] 3009    2

,cell,group
,<chr>,<chr>
28781,p009t2:GCAGCTGAGCTAGAAT,malignant_p009t2
28782,p009t2:GGGAGTATCAGCGCAC,malignant_p009t2
28783,p009t2:GTTAGACAGTACCCTA,malignant_p009t2
28784,p009t2:TTCACGCGTGGACTGA,malignant_p009t2
28785,p009t2:AAGGTAACACGAAAGC,malignant_p009t2
28786,p009t2:ATGGAGGTCTATCGGA,malignant_p009t2


In [27]:
# ? use all the normal

In [75]:
normal_anno_all <- anno[grepl('n',anno$cell),]
ref_internal_all <- aggregate_counts(normal_counts, normal_anno_all)
summary(ref_internal_all)

cell_dict
  epi 
13141 


      epi           
 Min.   :0.000e+00  
 1st Qu.:1.000e-08  
 Median :1.600e-07  
 Mean   :2.982e-05  
 3rd Qu.:5.540e-06  
 Max.   :8.983e-02  

normal_anno <- anno[grepl('n1',anno$cell),]


head(normal_anno)

dim(normal_anno)

ref_internal <- aggregate_counts(normal_counts, normal_anno)


summary(ref_internal)

normal_anno2 <- anno[grepl('n2',anno$cell),]
dim(normal_anno2)

ref_internal2 <- aggregate_counts(normal_counts, normal_anno2)


summary(ref_internal2)

#### merge ref_internal, and allele_df

ref_internal_avg =(ref_internal + ref_internal2)/2

In [79]:
merge_allele_df = rbind(allele_df, allele_df2)

#### p009_1 and p009_2 have a lot of conflicting GT
#### here I ran phasing together so should not have conflicting GT

# conflicting GT
conflicting_GT = merge_allele_df %>% group_by(snp_id) %>% mutate(n = length(unique(GT))) %>% filter(n >1)
dim(conflicting_GT)

dim(conflicting_GT)[1]/dim(merge_allele_df)[1]

sort(table(paste(conflicting_GT$CHROM,conflicting_GT$POS)),decreasing = T)

conflicting_GT %>% filter(CHROM == '9' && POS == '33110841') %>% filter(grepl('p009t2', cell)) %>% 
summarise(unique(GT))

In [80]:
dim(merge_allele_df)
head(merge_allele_df)

[1] 4206503      11

,cell,snp_id,CHROM,POS,cM,REF,ALT,AD,DP,GT,gene
,<chr>,<chr>,<int>,<int>,<dbl>,<chr>,<chr>,<int>,<int>,<chr>,<chr>
1,p009t1:ATTACCTAGATTGAGT,1_630110_T_C,1,630110,0,T,C,0,1,0|1,
2,p009t1:CTCAATTCAAGACGAC,1_630110_T_C,1,630110,0,T,C,0,1,0|1,
3,p009t1:GAACTGTCATGGCCCA,1_630110_T_C,1,630110,0,T,C,0,1,0|1,
4,p009t1:GGTCACGGTCACAGTT,1_630110_T_C,1,630110,0,T,C,1,1,0|1,
5,p009t1:GTATTGGGTCCAACGC,1_630110_T_C,1,630110,0,T,C,0,1,0|1,
6,p009t1:TCATTTGCACACTGGC,1_630110_T_C,1,630110,0,T,C,0,1,0|1,


In [ ]:
numbat_out <- run_numbat(
    tumor_counts, # gene x cell integer UMI count matrix
    ref_internal_all, # reference expression profile, a gene x cell type normalized expression level matrix
    merge_allele_df, # allele dataframe generated by pileup_and_phase script
    genome = "hg38",
    t = 1e-5,
    ncores = 4,
    plot = TRUE,
    out_dir = paste0('../../datasets_new_preprocessing/Numbat/', donor)
)



Running under parameters:
t = 1e-05
alpha = 1e-04
gamma = 20
min_cells = 50
init_k = 3
max_cost = 1819.8
max_iter = 2
max_nni = 100
min_depth = 0
use_loh = auto
multi_allelic = TRUE
min_LLR = 5
min_overlap = 0.45
max_entropy = 0.5
skip_nj = FALSE
diploid_chroms = 
ncores = 4
ncores_nni = 4
common_diploid = TRUE
tau = 0.3
check_convergence = FALSE
plot = TRUE
genome = hg38
Input metrics:
6066 cells

Mem used: 3.38Gb

Approximating initial clusters using smoothed expression ..

Mem used: 3.38Gb

number of genes left: 10034

